In [16]:
import json
import csv
import pandas as pd
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import spacy
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse
from nltk.sentiment import SentimentIntensityAnalyzer


nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/qianfu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/qianfu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/qianfu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Preprocessing the data

In [3]:
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # remove special characters, punctuation, and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # tokenize the text
    tokens = word_tokenize(text)
    # lemmatize and remove stopwords
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stopwords.words('english')]
    # join the tokens back into a string
    return ' '.join(lemmatized_tokens)


In [4]:
df = pd.read_csv('final_data.csv')

In [5]:
df['text'] = df['text'].apply(preprocess_text)

In [6]:
df['text'].value_counts()

text
xBnn                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

### further cleaning 

In [8]:
df = df[~df['text'].str.contains('xBnn|Title|^Please use thread|^ReadOnly Friday', case=False, na=False)]

In [9]:
df.shape

(7954, 17)

In [10]:
df2 = pd.read_csv('final_data.csv')

In [11]:
df2.shape

(8758, 17)

In [12]:
df['text'].value_counts()

text
CIS stand Computer information SystemsnnIve heard people I know I switch business different major within business school instead They told easier MISCIS expirencing mass layoff job market MISCIS extremely oversaturated They told tough job market MISCIS major even time I graduate itd better switch business MISCIS isnt good used What yall thinkn                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [13]:
df.drop_duplicates(subset='text', keep='first', inplace=True)


In [14]:
df.shape

(7863, 17)

In [15]:
df.to_csv('ready_data.csv', index=False)

### Performing Sentiment Analysis with VADER